In [48]:
import requests
import lxml.html
from lxml import etree

In [49]:
import time
import pprint
from collections import defaultdict

In [50]:
from pymongo import MongoClient

In [51]:
client = MongoClient('localhost', 27017)
db = client.geekreader
collection = db.leets

In [52]:
FORUM_URL = "https://discuss.leetcode.com"

In [53]:
answer_dict = defaultdict(list)

In [67]:
def get_doc_root(url):
    r = requests.get(url)
    html_text = r.text
    #my_parser = etree.HTMLParser(encoding='utf-8')
    #root = etree.HTML(html_text, parser=my_parser)
    return etree.HTML(html_text)
    #return root

In [82]:
def get_answer(url):
    url = "https://discuss.leetcode.com/topic/51893/two-solutions-one-is-dp-the-other-is-greedy-8-lines"
    print "getting", url
    root = get_doc_root(url)
    answer = {}
    content_ele = root.cssselect('div.answer div.content')[0]
    answer['url'] = url
    answer['content'] = etree.tostring(content_ele)
    username_ele = root.cssselect('div.user-info a')[0]
    answer['author_url'] = FORUM_URL + username_ele.attrib['href']
    answer['author'] = username_ele.text
    vote_ele = root.cssselect('div.vote-count')[0]
    answer['vote'] = int(vote_ele.text)
    tags_ele = root.cssselect('div.tags a span.tag-item')
    if not tags_ele:
        tags = []
    else:
        tags = [tag_ele.text for tag_ele in tags_ele]
    answer['tags'] = tags
    question_url_ele = root.cssselect('a#back_to_leetcode')[0]
    question_url = question_url_ele.attrib['href'] + "/"
    answer_dict[question_url].append(answer)

In [83]:
for line in open('urls.txt').readlines()[:1]:
    url = line.strip()
    time.sleep(3)
    get_answer(url)

getting https://discuss.leetcode.com/topic/51893/two-solutions-one-is-dp-the-other-is-greedy-8-lines
['c++', 'greedy', 'dynamic-programming']


In [73]:
for key in answer_dict:
    print answer_dict[key]
    # collection.update({url: key}, {'$set': {'answers': answer_dict[key]}})

[{'tags': ['\n\t\t', '\n\t\t', '\n\t\t'], 'url': 'https://discuss.leetcode.com/topic/51893/two-solutions-one-is-dp-the-other-is-greedy-8-lines', 'author': 'clubmaster', 'content': '<div class="content" component="post/content" itemprop="text">\n\t\t\t<p>DP:</p>\n<pre><code>class Solution {\npublic:\n    int wiggleMaxLength(vector&lt;int&gt;&amp; nums) {\n        int size=nums.size();\n        if(size==0) return 0;\n        vector&lt;int&gt; f(size, 1);\n        vector&lt;int&gt; d(size, 1);\n        for(int i=1; i&lt;size; ++i){\n            for(int j=0; j&lt;i; ++j){\n                if(nums[j]&lt;nums[i]){\n                    f[i]=max(f[i], d[j]+1);\n                }\n                else if(nums[j]&gt;nums[i]){\n                    d[i]=max(d[i], f[j]+1);\n                }\n            }\n        }\n        return max(d.back(), f.back());\n    }\n};\n</code></pre>\n<p>Greedy:</p>\n<pre><code>class Solution {\npublic:\n    int wiggleMaxLength(vector&lt;int&gt;&amp; nums) {\n      